In [ ]:
from hybridbrep import GeneralConvEncDec, BRepFaceAutoencoder, HybridPartDataset
import json
from zipfile import ZipFile
import torch
from tqdm import tqdm
import numpy as np
from matplotlib import pyplot as plt
from train_latent_space import BRepFaceAutoencoder as OldAutoencoder, BRepDS as OldDS
import altair as alt
import pandas as pd

In [ ]:
old_model = BRepFaceAutoencoder(64, 1024, 4, True)
new_model = GeneralConvEncDec(64, 1024, 4)

old_ckpt_path = '/home/ben/Documents/research/repbrep/training_logs/reconstruction/old_with_loops/version_0/checkpoints/epoch=54-val_loss=0.002403.ckpt'
new_ckpt_path = '/home/ben/Documents/research/repbrep/training_logs/reconstruction/new_with_edges/version_1/checkpoints/epoch=183-val_loss=0.002646.ckpt'

old_ckpt = torch.load(old_ckpt_path)
new_ckpt = torch.load(new_ckpt_path)

old_model.load_state_dict(old_ckpt['state_dict'])
new_model.load_state_dict(new_ckpt['state_dict'])

In [ ]:
from hybridbrep import HybridPartDataset

In [ ]:
ds = HybridPartDataset('../../datasets/fabwave.json', '../../datasets/fabwave_hpart_500_40.zip', '../../datasets/fabwave.zip', normalize=True, sort_frac=.4)

In [ ]:
with open('../../datasets/fabwave.json','r') as f:
    index = json.load(f)
train_names = [index['template'].format(*k) for k in index['train']]
test_names = [index['template'].format(*k) for k in index['test']]
names = train_names + test_names
with ZipFile('../../datasets/fabwave_hpart_500_40.zip','r') as zf:
    data = []
    for n in tqdm(names):
        with zf.open(n, 'r') as f:
            data.append((n, torch.load(f)))

In [ ]:
labels = {}

for i in range(len(index['train'])):
    k = index['train'][i]
    n = index['template'].format(*k)
    l = torch.tensor(index['train_labels'][i]).long()
    labels[n] = l

for i in range(len(index['test'])):
    k = index['test'][i]
    n = index['template'].format(*k)
    l = torch.tensor(index['test_labels'][i]).long()
    labels[n] = l

In [ ]:
new_precoded = {}
with torch.no_grad():
    for n, hdata in tqdm(data):
        y = labels[n]
        x = new_model.encode_faces(hdata).float()
        edge_index = hdata.face_to_face[:2].long()
        new_precoded[n] = {
            'x':x,
            'y':y,
            'edge_index':edge_index
        }

In [ ]:
torch.save(new_precoded,'../../datasets/fabwave_precoded_new_model_new_data.pt')